## This notebook shows the full workflow for building models, simulating growth and obtaining SCFA predictions from data collected by the _ex vivo_ study conducted by Gurry et al. 2021 (Study C)

In [1]:
import os
import pandas as pd
from plotnine import *
from tqdm import tqdm
import numpy as np
import scipy
import micom
import micom.measures

%matplotlib inline

## We'll start with the measured SCFA values from the _ex vivo_ experiments, and calculated production flux from the concentrations

In [2]:
# Read in the SCFA data
scfas = pd.read_csv('../data/studyC_SCFAs.csv',index_col = 0) 
# Melt the data into long form
scfas = pd.melt(scfas, 
                id_vars = ['subject_id','treatment','timepoint','replicate'], 
                value_vars = ['butyrate','propionate','acetate'],
                value_name = 'concentration',
                var_name = 'analyte')
# Sort values
scfas = scfas.sort_values(by = ['subject_id','treatment','analyte','timepoint','replicate'])
# Calculate the difference in concentration between timeoints
scfas['flux'] = scfas.groupby(['subject_id','treatment','replicate','analyte'])['concentration'].diff()
# Divide SCFA difference by culturing time to calculate flux
scfas['flux'] = scfas['flux']/4
# Drop rows with NAs
scfas.dropna(inplace = True)
# Find the mean of fluxes between replicates
res = scfas.groupby(['subject_id','treatment','analyte']).mean()
# Add the standard deviation for each group
res['sem']= scfas.groupby(['subject_id','treatment','analyte'])['flux'].sem()
# Reset the index of the dataframe keeping only samples that are used 
res = res.reset_index()[['subject_id','treatment','analyte','flux','sem']]
# Pivot the table for ease of plotting
measured = pd.pivot_table(res,columns = 'analyte',index = ['subject_id','treatment'],values = ['flux','sem'])
measured

flux                            sem            \
analyte                acetate  butyrate propionate   acetate  butyrate   
subject_id treatment                                                      
H008       CTRL       0.907748  0.072262   0.017734  0.005227  0.009684   
           INUL       0.630312  0.368432   0.213300  0.157356  0.023349   
           PECT       0.344484  0.135646   0.099018  0.256223  0.015573   
H009       CTRL       0.041150  0.054562   0.023126  0.008680  0.003392   
           INUL       0.033867  0.126382   0.030664  0.018637  0.001238   
           PECT       0.002474  0.081923   0.002512  0.005678  0.000961   
H010       CTRL       0.803625  0.044805   0.079866  0.803514  0.000117   
           INUL       2.530928  0.117080   0.085078  0.042526  0.005480   
           PECT       2.544074  0.047970   0.045062  0.039144  0.007849   
H012       CTRL      -0.036605  0.019954  -0.005870  0.024912  0.013323   
           INUL      -0.000431  0.140607   0.029837  0.017950  0.089164   
           PECT       0.022320  0.052147   0.005022  0.032617  0.033730   
H019       CTRL       0.269958  0.033968   0.005664  0.033320  0.005523   
           INUL       0.414354  0.521920   0.044275  0.003452  0.008081   
           PECT       0.387959  0.221448   0.036709  0.084427  0.017988   
H020       CTRL       1.010100  0.129339   0.050015  0.138052  0.004986   
           INUL       0.416795  0.212270   0.037061  0.072837  0.014843   
           PECT       0.505387  0.210041   0.034281  0.003205  0.019756   
H021       CTRL       0.290202  0.039797   0.005905  0.022226  0.000063   
           INUL       0.248695  0.440074   0.028125  0.002957  0.035062   
           PECT       0.336364  0.213509   0.022763  0.036704  0.020580   
H025       CTRL       0.320694  0.213345   0.045562  0.088369  0.028460   
           INUL       0.467728  0.585425   0.120354  0.046235  0.013323   
           PECT       0.173211  0.424253   0.059136  0.096065  0.011054   
H028       CTRL       0.074091  0.076664   0.054135  0.014500  0.011425   
           INUL       0.778718  0.346605   0.125184  0.060204  0.001158   
           PECT       0.591876  0.178355   0.151254  0.088268  0.004083   

                                 
analyte              propionate  
subject_id treatment             
H008       CTRL        0.016793  
           INUL        0.003687  
           PECT        0.014555  
H009       CTRL        0.001578  
           INUL        0.000215  
           PECT        0.000562  
H010       CTRL        0.001090  
           INUL        0.006000  
           PECT        0.011373  
H012       CTRL        0.005237  
           INUL        0.018871  
           PECT        0.004111  
H019       CTRL        0.003514  
           INUL        0.004425  
           PECT        0.010165  
H020       CTRL        0.000665  
           INUL        0.008636  
           PECT        0.015760  
H021       CTRL        0.001278  
           INUL        0.000668  
           PECT        0.000021  
H025       CTRL        0.013834  
           INUL        0.000451  
           PECT        0.007281  
H028       CTRL        0.008819  
           INUL        0.013712  
           PECT        0.001179

## Next, we'll load up abundances and build a taxonomy table for MICOM

In [3]:
taxonomy = pd.read_csv('../data/studyC_abundances.csv')
# Melt the table into long form
taxonomy = pd.melt(taxonomy, 
                   id_vars = 'sample_id', 
                   value_vars = taxonomy.columns[1:], 
                   var_name = 'id', 
                   value_name = 'abundance')
# Add a genus column, as required by MICOM for the taxonomy table
taxonomy['genus'] = taxonomy['id'].str.split(';').str[5].str.split('.').str[0]
taxonomy

,sample_id,id,abundance,genus
0,H008,Bacteria;Firmicutes;Clostridia;Clostridiales;R...,49.0,Faecalibacterium
1,H009,Bacteria;Firmicutes;Clostridia;Clostridiales;R...,133.0,Faecalibacterium
2,H010,Bacteria;Firmicutes;Clostridia;Clostridiales;R...,43.0,Faecalibacterium
3,H012,Bacteria;Firmicutes;Clostridia;Clostridiales;R...,456.0,Faecalibacterium
4,H019,Bacteria;Firmicutes;Clostridia;Clostridiales;R...,32.0,Faecalibacterium
...,...,...,...,...
14315,H020,Bacteria;Firmicutes;Clostridia;Clostridiales;L...,0.0,[eubacterium]
14316,H021,Bacteria;Firmicutes;Clostridia;Clostridiales;L...,1.0,[eubacterium]
14317,H025,Bacteria;Firmicutes;Clostridia;Clostridiales;L...,0.0,[eubacterium]
14318,H028,Bacteria;Firmicutes;Clostridia;Clostridiales;L...,0.0,[eubacterium]


## We need a model database to pull our reconstructions from to build models

In [4]:
agora = ('../agora/data')

## Now, we'll build our models, with cutoff of 0.001

In [5]:
models = micom.workflows.build(taxonomy,out_folder = '../models/studyC',
                      model_db = agora, cutoff = 0.001, threads = 10)

[16:26:25] WARNING  Found existing models for 10 samples. Will skip those. Delete the output folder if  ]8;id=994854;file:///users/nbohmann/.local/lib/python3.8/site-packages/micom/workflows/build.py\build.py]8;;\:]8;id=31299;file:///users/nbohmann/.local/lib/python3.8/site-packages/micom/workflows/build.py#95\95]8;;\
                    you would like me to rebuild them.                                                             

Output()

## Next, we'll gather our medium. We'll load in the carbon-depleted standard european medium that was constructed and functionally completed in the medium_construction.ipynb notebook.

In [6]:
# Read in the medium
medium = pd.read_csv('../media/european_lowcarbon.csv',index_col = 0)[['reaction','metabolite','flux']]
# Dilute to 10% of original content
medium['flux'] = medium['flux']*.1
# We'll also build two treatment media, including an inulin supplement or a pectin supplement
pectin = pd.DataFrame({'reaction':['EX_pect_m'],'metabolite':['pect_m'],'flux':[0.75]})
inulin = pd.DataFrame({'reaction':['EX_inulin_m'],'metabolite':['inulin_m'],'flux':[10.50]})
pectin_medium = pd.concat([medium, pectin])
inulin_medium = pd.concat([medium, inulin])

## Now, we'll use the diet to grow the models we just built. First, we'll do so without the addition of fiber, to simulate control samples 

In [7]:
# Read the model manifest
manifest = pd.read_csv('../models/studyC/manifest.csv')
# Grow the models with a tradeoff value of 0.7
growth = micom.workflows.grow(manifest,'../models/studyC',medium=medium,tradeoff=.7,strategy='none',threads = 20)
# # Collect the absolute production rates
control = micom.measures.production_rates(growth)

Output()

/users/nbohmann/.local/lib/python3.8/site-packages/micom/workflows/grow.py:199: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


## We'll do the same with the supplementation of pectin, on the treated samples

In [ ]:
# Read the model manifest
manifest = pd.read_csv('../models/studyC/manifest.csv')
# Grow the models with a tradeoff value of 0.7
growth = micom.workflows.grow(manifest,'../models/studyC',medium=pectin_medium,tradeoff=.7,strategy='none',threads = 20)
# Collect the absolute production rates
pectin = micom.measures.production_rates(growth)

Output()

## And again with inulin

In [ ]:
# Read the model manifest
manifest = pd.read_csv('../models/studyC/manifest.csv')
# Grow the models with a tradeoff value of 0.7
growth = micom.workflows.grow(manifest,'../models/studyC',medium=inulin_medium,tradeoff=.7,strategy='none',threads = 20)
# Collect the absolute production rates
inulin = micom.measures.production_rates(growth)

## Now that we have the results, we'll isolate the SCFA production and format it for concatenation with the measured results. Here's a function that will do this for both the control and inulin treated samples. 


In [ ]:
def get_fluxes(df, treatment):
    # Isolate SCFA fluxes
    df = df[(df['metabolite']=='but[e]')|(df['metabolite']=='ppa[e]')|(df['metabolite']=='ac[e]')]
    # Map the metadata onto the dataframe to align with the measured samples 
    df['subject_id'] = df['sample_id']
    df['treatment'] = treatment
    # Take the average production of the replicates
    res = df.groupby(['subject_id','treatment','name']).mean().reset_index()
    
    # Pivot the table for alignment with the measurement dataframe
    predicted = pd.pivot_table(res,
                columns = 'name',
                index = ['subject_id','treatment'], 
                values = 'flux')
    predicted = predicted.rename(columns = {'acetate':'flux-predicted_acetate',
                                'butyrate':'flux-predicted_butyrate',
                                'propionate':'flux-predicted_propionate'}).reset_index()
    return predicted

## Now we can iterate the function over both the control and treatment results, concatenating them into one dataframe

In [ ]:
# Control Samples
predicted = get_fluxes(control, 'CTRL',)
# Pectin samples
predicted = pd.concat([predicted, get_fluxes(pectin, 'PECT')], axis = 0)
# And inulin samples
predicted = pd.concat([predicted, get_fluxes(inulin, 'INUL')], axis = 0)
predicted.set_index(['subject_id','treatment'], inplace = True)

## Now we'll concatenate the predictions with the measured scfa values, and rename the columns so they don't overlap

In [ ]:
measured.columns = ['_'.join(col) for col in measured.columns.values]
# Concatenate dataframes
scfasTotal = pd.concat([measured,predicted], axis = 1)
# Flatten multiindex
scfasTotal.columns = scfasTotal.columns.to_flat_index()
# Join column names for plotting
scfasTotal.reset_index(inplace = True)

In [ ]:
scfasTotal

## Finally, plot the results

In [ ]:
ac_plot = (ggplot(
    scfasTotal, aes(x = ('flux_acetate'), y = ('flux-predicted_acetate')))
    +geom_point(aes(color = 'treatment'),size = 5)
    +geom_errorbarh(aes(y ="flux-predicted_acetate", 
                        xmin = scfasTotal['flux_acetate'] - scfasTotal['sem_acetate'],
                        xmax = scfasTotal['flux_acetate'] + scfasTotal['sem_acetate']))
    +geom_smooth(method='lm',linetype='--')
    +scale_color_manual(values = ['deepskyblue','darksalmon','lightgreen'])
    +labs(x='Measured($\dfrac{mmol}{L*h}$)',y = 'Predicted($\dfrac{mmol}{gDCW*h}$)',title='Acetate')
    +theme(text = element_text(size=20),panel_background=element_rect(fill = "white",
                            colour = "white",size = 0.5, linetype = "solid"),
                            panel_grid=element_line(size = .2, linetype = "solid",colour = "gray"),
                            axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                            legend_title=element_blank(),legend_position='right'))
ac_plot

In [ ]:
ppa_plot = (ggplot(
    scfasTotal, aes(x = ('flux_propionate'), y = ('flux-predicted_propionate')))
    +geom_point(aes(color = 'treatment'),size = 5)
    +geom_errorbarh(aes(y = 'flux-predicted_propionate', 
                        xmin = scfasTotal['flux_propionate'] - scfasTotal['sem_propionate'],
                        xmax = scfasTotal['flux_propionate'] + scfasTotal['sem_propionate']))
    +geom_smooth(method='lm',linetype='--')
    +scale_color_manual(values = ['deepskyblue','darksalmon','lightgreen'])
    +labs(x='Measured($\dfrac{mmol}{L*h}$)',y = 'Predicted($\dfrac{mmol}{gDCW*h}$)',title='Propionate')
    +theme(text = element_text(size=20),panel_background=element_rect(fill = "white",
                            colour = "white",size = 0.5, linetype = "solid"),
                            panel_grid=element_line(size = .2, linetype = "solid",colour = "gray"),
                            axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                            legend_title=element_blank(),legend_position='right'))
ppa_plot

In [ ]:
but_plot = (ggplot(
    scfasTotal, aes(x = ('flux_butyrate'), y = ('flux-predicted_butyrate')))
    +geom_point(aes(color = 'treatment'),size = 5)
    +geom_errorbarh(aes(y ='flux-predicted_butyrate', 
                        xmin = scfasTotal['flux_butyrate'] - scfasTotal['sem_butyrate'],
                        xmax = scfasTotal['flux_butyrate'] + scfasTotal['sem_butyrate']))
    +geom_smooth(method='lm',linetype='--')
    +scale_color_manual(values = ['deepskyblue','darksalmon','lightgreen'])
    +labs(x='Measured($\dfrac{mmol}{L*h}$)',y = 'Predicted($\dfrac{mmol}{gDCW*h}$)',title='Butyrate')
    +theme(text = element_text(size=20),panel_background=element_rect(fill = "white",
                            colour = "white",size = 0.5, linetype = "solid"),
                            panel_grid=element_line(size = .2, linetype = "solid",colour = "gray"),
                                  axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                            legend_title=element_blank(),legend_position='right'))
but_plot

## Save the results for future use

In [ ]:
scfasTotal.to_csv('../data/studyC.csv')